In [88]:
import pandas as pd, numpy as np

import ynet

import requests

import bs4

import json

In [10]:
valid_topic_ids = ['891-317', '109-185']

def _scrape_articles_urls_from_topic(topic_id: str, year: int, month: int):
    if year < 2000:
        year = 2000 + year if year < 30 else 1900 + year
    if month < 10:
        month = '0' + str(month)
    url = f'https://www.ynet.co.il/home/0,7340,L-4269-{topic_id}-{year}{month}-1,00.html'
    html = bs4.BeautifulSoup(requests.get(url).content)
    html = html.find_all(attrs={'class': 'ghciArticleIndex1'})[0].find_all('table')[1]

    articles_htmls = html.find_all(attrs={'class': 'smallheader'})
    articles_urls = ['https://www.ynet.co.il' + article.get('href')
                    for article in articles_htmls]
    return articles_urls

def scrape_articles():
    articles_urls = []
    for topic_id in valid_topic_ids:
        for year in range(2000,2021 + 1):
            for month in range(1,12 + 1):
                try:
                    urls = _scrape_articles_urls_from_topic(topic_id, year, month)
                    assert type(urls) == list
                    articles_urls += urls
                except:
                    print(topic_id, year, month, 'failed')
                    pass
    return list(set(articles_urls))

In [11]:
articles_urls = scrape_articles()

891-317 2021 7 failed
109-185 2001 8 failed
109-185 2002 4 failed
109-185 2004 5 failed
109-185 2004 11 failed
109-185 2016 6 failed
109-185 2016 7 failed
109-185 2016 8 failed
109-185 2016 9 failed
109-185 2016 10 failed
109-185 2016 11 failed
109-185 2016 12 failed
109-185 2017 1 failed
109-185 2017 2 failed
109-185 2017 3 failed
109-185 2017 4 failed
109-185 2017 5 failed
109-185 2017 6 failed
109-185 2017 7 failed
109-185 2017 8 failed
109-185 2017 9 failed
109-185 2017 10 failed
109-185 2017 11 failed
109-185 2017 12 failed
109-185 2018 1 failed
109-185 2018 2 failed
109-185 2018 3 failed
109-185 2018 4 failed
109-185 2018 5 failed
109-185 2018 6 failed
109-185 2018 7 failed
109-185 2018 8 failed
109-185 2018 9 failed
109-185 2018 10 failed
109-185 2018 11 failed
109-185 2018 12 failed
109-185 2019 1 failed
109-185 2019 2 failed
109-185 2019 3 failed
109-185 2019 4 failed
109-185 2019 5 failed
109-185 2019 6 failed
109-185 2019 7 failed
109-185 2019 8 failed
109-185 2019 9 failed


In [14]:
def save_list(l):
    with open('articles_urls.txt', 'w') as f:
        for item in l:
            f.write("%s\n" % item)
save_list(articles_urls)

In [97]:
article_url.replace('http', 'htt')

'htt://www.ynet.co.il/articles/0,7340,L-3981508,00.html'

In [122]:
def get_article(article_url: str):
    try:
        if article_url.startswith('https'):
            article_url = article_url.replace('https', 'http')
        print(article_url)
        html = bs4.BeautifulSoup(requests.post(article_url, timeout=(100.05, 1000)).content)
        ld_json = html.find_all(attrs={'type' : 'application/ld+json'})[0]
        ld_json = json.loads([l for l in ld_json.children][0])
        main_title = ld_json['headline']
        sub_title = ld_json['description']
        content = ld_json['articleBody']

        # main_title = html.find_all(attrs={'class' : 'mainTitle'})[0].text.strip('"')
        # sub_title = html.find_all(attrs={'class' : 'subTitle'})[0].text.strip('"')
        # content = html.find_all(attrs={'class' : 'public-DraftEditor-content'})[0].text.strip('"')
        return pd.Series({
            'main_title' : main_title, 
            'sub_title' : sub_title, 
            'content' : content
        })
    except:
        print(f'{article_url} failed')
        return None

In [115]:
urls = pd.Series(np.random.choice(articles_urls, 5))
data = urls.apply(get_article)

http://www.ynet.co.il/articles/0,7340,L-3372849,00.html
http://www.ynet.co.il/articles/0,7340,L-5457140,00.html
http://www.ynet.co.il/articles/0,7340,L-4706007,00.html
http://www.ynet.co.il/articles/0,7340,L-5719869,00.html
http://www.ynet.co.il/articles/0,7340,L-3166897,00.html


In [131]:
content.apply(print)

הסקרים שפורסמו היום הכניסו את המתמודדים על ראשות העבודה ללהיטות יתר. שר הביטחון, עמיר פרץ, שהגיע למקום השלישי, מאמין שעבודה נכונה ביום הבחירות תשאיר אותו בתפקיד. במטהו של ברק סופרים קולות ואומרים: "עוד 26 אלף מתפקדים וננצח בסיבוב הראשון"
בלילה אמרו בליכוד כי לרה"מ אין רשימת מומלצים לפריימריז, הבוקר מסר נתניהו כי "רשימת המומלצים היא כל הח"כים". המועמד היחיד שהזכיר בשמו הוא דוד שרן, ראש הסגל לשעבר שהמשטרה הגישה המלצות להאשימו בפרשת הצוללות. "מאוד מעריך אותו", אמר נתניהו
פרסום ראשון: הוועדה למינוי בכירים אישרה בצהריים את מינויו של סגן ראש השב"כ רוני אלשיך. השלב הבא: אישור הממשלה. מוקדם יותר אמר השר ארדן: "שמחתי לשמוע שלוועדה לא הגיע אפילו מכתב התנגדות אחד נגד המינוי"
בצל ביקורת קשה על תפקודו במשבר הקורונה, שוקל שר הבריאות לעבור למשרד השיכון - שבו האינטרס החרדי גדול יותר - ואף שוחח בנושא עם ראש הממשלה נתניהו ועם נשיא מועצת גדולי התורה. בסביבתו הבהירו: טרם התקבלה החלטה
עוד לפני שנסתיימה ספירת הקולות, יצא פרס העייף אל העיתונאים ואמר כי יש דברים "תמוהים ביותר" בהתנהלות הפריימריז. "יש תלונות מ

0    None
1    None
2    None
3    None
4    None
Name: sub_title, dtype: object

In [137]:
data['content'].apply(print)

מעבירים הילוך בקרב על ראשות מפלגת העבודה. סקרי דעת הקהל הראשונים שפורסמו הבוקר (יום ב') הכניסו את מטות כל המועמדים בפריימריז לראשות מפלגת העבודה ללהיטות יתר. אהוד ברק, אופיר פינס, עמי אילון ודני יתום הפגינו שביעות רצון מתוצאות הסקרים. בסביבתו הקרובה של יו"ר מפלגת העבודה, השר עמיר פרץ, מיהרו להבהיר כי אין בכוונתו להסיר את מועמדותו ולחבור למועמד אחר.   לפי סקר "דחף" שנערך עבור "ידיעות אחרונות" עולה כי לא ניתן לקבוע בוודאות מי באמת מוביל: עמי אילון זוכה ל-28%, אהוד ברק - 26%, עמיר פרץ - 19%, אופיר פינס - 17% ודני יתום ל-6%. פרץ אמר ל-ynet: "אין לי שום כוונה לחבור לאיש מהמועמדים או לשקול פרישה מהמערכת הפוליטית. אני כאן כדי להישאר ואני מתכוון לנצח". פרץ הוסיף כי הוא שמח על כך שהגיע למקום השלישי בסקרים. לדבריו, כך אף אחד מהמועמדים לא ירצה לחבור למועמד אחר. "יהיה מירוץ בו אנצח. לא צריך להאמין לכל מה שאומרים גורמים עלומי שם המנסים להפיץ שקרים".   במטה פרץ מעריכים כי אם יגיעו כשישים אלף חברי מפלגה להצביע, הרי שיו"ר העבודה יוכל לנצח באמצעות הבאתם של עשרים אלף חברי מפלגה לקלפיות. "גם אם נביא רק א

0    None
1    None
2    None
3    None
4    None
Name: content, dtype: object

In [132]:
titles.apply(print)

פרץ: ארוץ עד הסוף בפריימריז
נתניהו לצד החשוד מפרשת הצוללות: "מועמד ראוי ביותר"
סוף הסאגה: טירקל אישר מינוי אלשיך למפכ"ל
ליצמן שוקל לוותר על תיק הבריאות ולעבור לשיכון
שמעון פרס: "לבדוק חשדות לזיופים"


0    None
1    None
2    None
3    None
4    None
Name: main_title, dtype: object

In [130]:
content = data['sub_title']
titles = data['main_title']

In [118]:
X = content.str.split()
y = titles.str.split()
X.sample()

2    [פרסום, ראשון:, הוועדה, למינוי, בכירים, אישרה,...
Name: sub_title, dtype: object

In [143]:
data.apply(lambda r: [(r['content'].split().index(word) if word in r['content'].split() else -1) for word in r['sub_title'].split()], axis = 1)

0    [259, 11, 261, 15, 16, -1, 3, 4, 23, 24, 25, 2...
1    [-1, -1, 169, 19, -1, 40, 42, 43, -1, 16, -1, ...
2    [-1, -1, 195, 9, 19, 22, 23, 26, 27, 28, 29, 3...
3    [-1, 375, 376, 16, 264, -1, -1, 61, 0, 1, -1, ...
4    [5, 6, -1, 127, 128, -1, 12, -1, -1, 162, -1, ...
dtype: object